In [1]:
import lucem_illud #pip install -U git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git

#All these packages need to be installed from pip
#For NLP
import nltk

import numpy as np #For arrays
import pandas #Gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import seaborn #Makes the graphics look nicer

#Displays the graphs
import graphviz #You also need to install the command line graphviz

#These are from the standard library
import os.path
import zipfile
import subprocess
import io
import tempfile
import re
import time

In [2]:
#news_df = pandas.concat([pandas.read_pickle('../news_df_norm1.pkl'), pandas.read_pickle('../news_df_norm2.pkl')])
news_df = pandas.read_pickle('../news_df_norm1.pkl')

In [3]:
#Unnecessary now
#lucem_illud.setupStanfordNLP()

In [2]:
import lucem_illud.stanford as stanford

/software/Anaconda3-5.0.0.1-el7-x86_64/lib/python3.6/site-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)
/software/Anaconda3-5.0.0.1-el7-x86_64/lib/python3.6/site-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


In [5]:
news_df['text'] = news_df['text'].apply(lambda x: re.sub(r'\.\s*"\s?', '". ', x))
news_df['text'] = news_df['text'].apply(lambda x: re.sub(r',\s*"\s?', '", ', x))
# Substitutes .A with . A
news_df['text'] = news_df['text'].apply(lambda x: re.sub(r'\.(?=[A-Z])', '", ', x))

In [6]:
news_df['tokenized_sentences'] = news_df['text'].apply(lambda x: [nltk.word_tokenize(s) for s in nltk.sent_tokenize(x)])

In [7]:
errors_out = []
oserrors = 0
news_df['parse_sents'] = np.nan
small_df = news_df.sample(10)

In [3]:
def set_parsed_list_or_nan(row, tokenized_sentences_col, id_col = None, error_list = None):
    from numpy import nan
    try:
        print("{} {}".format(row.name, row[id_col]))
        return [list(parsed) for parsed in stanford.parser.parse_sents(row[tokenized_sentences_col])]
    except ValueError as err:
        print(row[id_col])
        print(err)
        if error_list:
            error_list.append((row[id_col], "{}".format(err)))
        return nan
    except OSError as err:
        print(row[id_col])
        print(err)
        if error_list:
            error_list.append((row[id_col], "{}".format(err)))
        return nan

In [9]:
small_df['parse_sents'] = small_df.apply(lambda x: set_parsed_list_or_nan(x, 'tokenized_sentences', id_col='filename', error_list=errors_out), axis=1)

1417 ./data/CH/CH4_59.html
1964 ./data/IN/IN3_42.html
642 ./data/AU/AUS3_70.html
1532 ./data/CH/CH4_42.html
973 ./data/AU/AUS2_67.html
1693 ./data/CH/CH2_3.html
608 ./data/AU/AUS5_36.html
2065 ./data/IN/IN6_6.html
807 ./data/AU/AUS_16.html
754 ./data/AU/AU6_45.html


In [10]:
%matplotlib inline

In [106]:
import pdb

extract_direct_object(experiment)

''

In [105]:
# Look at 14
experiment = small_df.iloc[0].parse_sents[25][0][0][1][2][1][1][2][1]
experiment.pretty_print()
#print(get_SVOs_in_sentence_tree(experiment))

                       S                            
      _________________|_____                        
     |                       VP                     
     |             __________|___                    
     |            |             ADJP                
     |            |     _________|____               
     |            |    |              S             
     |            |    |              |              
     |            |    |              VP            
     |            |    |      ________|___           
     |            |    |     |            VP        
     |            |    |     |    ________|___       
     NP           |    |     |   |            NP    
  ___|_____       |    |     |   |         ___|___   
 DT        NN    VBD   JJ    TO  VB      PRP      ''
 |         |      |    |     |   |        |       |  
the     hospital was unable  to pay      him      ''



In [70]:
experiment[0,0,1,1].pretty_print()

                                                        VP                                                
    ____________________________________________________|____                                              
   |                                                         NP                                           
   |            _____________________________________________|____________________                         
   |           |             |                    |               |              SBAR                     
   |           |             |                    |               |     __________|_____                   
   |           |             |                    |               |    |                S                 
   |           |             |                    |               |    |           _____|_________         
   |           |             |                    |               |    |          |               VP      
   |           |             |   

In [70]:
for a in experiment[0].subtrees():
    if a.label()=='S':
        a.pretty_print()

                                                                 S                                                                                  
                   ______________________________________________|________________________________________________________________________________   
                  |                      VP                                                                                                       | 
                  |                   ___|_____                                                                                                   |  
                  |                  |         VP                                                                                                 | 
                  |                  |    _____|____                                                                                              |  
                  |                  |   |          VP                                                 

In [4]:
def extract_noun(tree):
    if not tree.label().startswith('N'):
        return ''
    result = ''
    for child in tree:
        if child.label() == 'NP':
            result += extract_noun(child)
        elif child.label().startswith('NN') or \
            child.label() == 'PRP':
            result += child[0] + ' '
        elif child.label() == ',':
            break
    return result

def extract_verb(tree, stem_verb=True, lemmer=None):
    if not tree.label().startswith('V'):
        return ''
    lemmer = nltk.stem.WordNetLemmatizer()
    result = ''
    for child in tree:
        if child.label() == 'VP':
            result += extract_verb(child, stem_verb=stem_verb, lemmer=lemmer)
        elif child.label().startswith('VB'):
            result += lemmer.lemmatize(child[0], pos='v') + ' '
    return result

def extract_direct_object(tree):
    #pdb.set_trace()
    if not tree.label().startswith('VP'):
        return ''
    for child in tree:
        if child.label() == 'NP':
            return extract_noun(child)
        elif child.label() == 'PP':
            for subchild in child:
                if subchild.label() == 'NP':
                    return extract_noun(subchild)
    for child in tree:
        if child.label() == 'VP':
            return extract_direct_object(child)
    return ''

def extract_SVO(tree, last_noun = True):
    subject = verb = objec = ''
    if not tree.label() == 'S':
        return None
    for child in tree:
        if child.label() == 'NP':
            subject = extract_noun(child)
        elif child.label() == 'VP':
            verb = condense_verbs(extract_verb(child))
            objec = extract_direct_object(child)
    if last_noun:
        subject = subject.strip().split(' ')[-1]
        objec = objec.strip().split(' ')[-1]
    return (subject, verb, objec)


In [162]:
lemming = nltk.stem.WordNetLemmatizer()
lemming.lemmatize('hidden', pos='v')

'hide'

In [5]:
def get_SVOs_in_sentence_tree(tree):
    result = []
    for subt in tree.subtrees():
        if subt.label()=='S':
            result.append(extract_SVO(subt, last_noun=True))
    return result

def condense_verbs(verbs):
    vblist = verbs.split()
    if len(vblist) < 2:
        return verbs
    aux= ['have', 'has', 'had', 'been', 'are', 'is', 'be', 'am', 'does', 'did', 'was', 'being', 'having']
    return " ".join([vb for vb in vblist if vb not in aux])

In [165]:
small_df = sampled_df
svo_big_list = []
for ix, row in small_df.iterrows():
    for sent_tree in row.parse_sents:
        for svo in get_SVOs_in_sentence_tree(sent_tree[0]):
            if svo[0] != '' : 
                svo_big_list.append({'filename': row['filename'],
                                'country': row['country'],
                                'subject': svo[0],
                                'verb': svo[1],
                                'object': svo[2]})
small_df['svos'] = small_df['parse_sents'].apply(lambda x: [get_SVOs_in_sentence_tree(i[0]) for i in x])

In [166]:
ix = 0
print(small_df.iloc[ix].text)
print(small_df.iloc[ix].svos)

 Angolans cast their ballots Wednesday in an election marking the end of President Jose Eduardo Dos Santos's 38-year reign, with his MPLA party set to retain power despite an economic crisis.  The MPLA, which has ruled since Angola's independence from Portugal in 1975, is expected to defeat opposition parties, which are stifled by Dos Santos's authoritarian regime.  Dos Santos's unexpected retirement — reportedly prompted by poor health — has triggered the biggest political transition in decades for Angola, a leading oil exporter in Africa.  However, his chosen successor is Defence Minister Joao Lourenco, a loyalist expected to avoid immediate change in a government often criticised for corruption and its failure to tackle dire poverty". My mission will be to revive the economy", Lourenco told reporters in the capital Luanda on the eve of the vote". If I succeed, I would like to be recognised in history as the man of Angola's economic miracle".  Dos Santos's long reign has seen the end

In [167]:
svo_df = pandas.DataFrame(svo_big_list)


In [168]:
len(svo_df)

45048

In [169]:
svo_df

,country,filename,object,subject,verb
0,KE,./data/KE/KE5_39.html,ballots,Angolans,cast
1,KE,./data/KE/KE5_39.html,,MPLA,expect
2,KE,./data/KE/KE5_39.html,transition,—,trigger
3,KE,./data/KE/KE5_39.html,Lourenco,successor,be
4,KE,./data/KE/KE5_39.html,reporters,Lourenco,tell
5,KE,./data/KE/KE5_39.html,,mission,be
6,KE,./data/KE/KE5_39.html,man,I,like
7,KE,./data/KE/KE5_39.html,,I,succeed
8,KE,./data/KE/KE5_39.html,end,reign,see
9,KE,./data/KE/KE5_39.html,benefit,flood,bring


In [170]:
svo_df.groupby(by='subject').count().sort_values(by='verb', ascending=False)

,country,filename,object,verb
subject,,,,
it,2334,2334,2334,2334
we,2043,2043,2043,2043
he,1463,1463,1463,1463
I,1444,1444,1444,1444
they,1163,1163,1163,1163
It,1097,1097,1097,1097
We,827,827,827,827
’,784,784,784,784
change,653,653,653,653


In [6]:
sampled_df = pandas.read_pickle('sampletrees3k.pkl')#'sampletrees1k.pkl')
sampled_df.dropna(inplace=True)
svo_big_list = []
import math
for ix, row in sampled_df.iterrows():
    for sent_tree in row['parse_sents']:
        for svo in get_SVOs_in_sentence_tree(sent_tree[0]):
            if svo[0] != '' : 
                svo_big_list.append({'filename': row['filename'],
                                'country': row['country'],
                                'subject': svo[0],
                                'verb': svo[1],
                                'object': svo[2]})
svo_df = pandas.DataFrame(svo_big_list)
#svo_df.to_pickle('svo_df.pkl')

In [7]:
svo_df

,country,filename,object,subject,verb
0,CH,./data/CH/CH2_38.html,networks,vision,evoke
1,CH,./data/CH/CH2_38.html,,regions,include
2,CH,./data/CH/CH2_38.html,,probability,dismiss anticipate
3,CH,./data/CH/CH2_38.html,series,situations,arise do occur
4,CH,./data/CH/CH2_38.html,framework,Goals,serve
5,CH,./data/CH/CH2_38.html,,key,be
6,CH,./data/CH/CH2_38.html,,they,use
7,CH,./data/CH/CH2_38.html,part,management,be
8,CH,./data/CH/CH2_38.html,conflicts,communities,resolve become exacerbate
9,CH,./data/CH/CH2_38.html,,they,arise


In [134]:
svo_df[svo_df['verb']=='said'].groupby('country')['verb'].value_counts()

country  verb
AU       said    12
CH       said    14
IN       said    12
KE       said     1
ME       said    12
NG       said    15
UK       said    10
US       said    10
Name: verb, dtype: int64

In [168]:
sampled_df[sampled_df['filename']=='./data/UK/UK1_23.html'].parse_sents[5511][7][0][0][2].pretty_print()

       VP                                                                                                                                                                                                                      
  _____|_______                                                                                                                                                                                                                 
 |             VP                                                                                                                                                                                                              
 |      _______|_________________                                                                                                                                                                                               
 |     |                        SBAR                                                                  

In [171]:
svo_df.groupby(by=['verb']).country.value_counts().sort_values(ascending=False)

verb         country
be           US         1356
             ME         1248
             CH         1077
             IN          932
             AU          771
             NG          691
say          US          631
be           UK          587
             KE          514
say          CH          513
             ME          487
s            ME          354
say          IN          352
             NG          346
             UK          327
             AU          291
             US          271
             ME          248
have         US          206
             CH          201
say          KE          198
have         CH          196
             ME          195
             IN          186
have         NG          172
             NG          164
s            US          159
's           US          148
have         AU          142
             IN          141
                        ... 
turf         AU            1
lag          AU            1
last         AU       

In [8]:
svo_df.to_pickle('svo_simplified.pkl')